In [ ]:
import jax
import jax.numpy as jnp
from jax.experimental.shard_map import shard_map
from jax.sharding import NamedSharding, PartitionSpec

In [ ]:
mesh = jax.make_mesh((4, 2), ('i', 'j'))

In [ ]:
x = jnp.array([[3.]])

In [ ]:
z = shard_map(lambda: x, mesh=mesh, in_specs=(), out_specs=PartitionSpec('i', 'j'))()
print(z)  # prints the same as jnp.tile(x, (4, 2))

z = shard_map(lambda: x, mesh=mesh, in_specs=(), out_specs=PartitionSpec('i', None))()
print(z)  # prints the same as jnp.tile(x, (4, 1)), or just jnp.tile(x, (4,))

z = shard_map(lambda: x, mesh=mesh, in_specs=(), out_specs=PartitionSpec(None, None))()
print(z)  # prints the same as jnp.tile(x, (1, 1)), or just x

In [ ]:
x = jnp.arange(8).reshape(4, 2)
x

In [ ]:
z = shard_map(lambda x: x, mesh=mesh, in_specs=PartitionSpec('i', 'j'), out_specs=PartitionSpec('i', 'j'))(x)
print(z)

z = shard_map(lambda x: x, mesh=mesh, in_specs=PartitionSpec('i', None), out_specs=PartitionSpec('i', None))(x)
print(z)

z = shard_map(lambda x: x, mesh=mesh, in_specs=PartitionSpec('i', None), out_specs=PartitionSpec(None, 'i'))(x)
print(z)

z = shard_map(lambda x: x, mesh=mesh, in_specs=PartitionSpec('i', None), out_specs=PartitionSpec('i', 'j'))(x)
print(z)

# `ppermute`

In [ ]:
def f(x):
  return jax.lax.ppermute(x, axis_name='i', perm=[(0, 1), (1, 2), (2, 3), (3, 0)])

x = jnp.arange(4)
z = shard_map(f, mesh=mesh, in_specs=PartitionSpec('i'), out_specs=PartitionSpec('i'))(x)
z

# `axis_index`

def f():
  return jnp.array([[jax.lax.axis_index('i'), jax.lax.axis_index('j')]])

z = shard_map(f, mesh=mesh, in_specs=PartitionSpec('i', 'j'), out_specs=PartitionSpec('i', 'j'))()
z